In [1]:
import pandas as pd
import numpy as np
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed 
from tqdm import tqdm
import os

/Library/Python/3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
dis = np.load('/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/dis_matrix.npy')
node = np.load('/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/node_order.npy')
sensor = pd.read_csv('/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/sensor_meta_feature.csv', sep = '\t')

In [3]:
dis_test = np.vsplit(dis, 4243)

In [4]:
def calculate_distance(lat1, lng1, lat2, lng2):
    url = f"http://192.168.221.197:5000/route/v1/driving/{lng1},{lat1};{lng2},{lat2}?overview=false"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['routes']:
            return data['routes'][0]['distance']
    return None  # Falls keine Verbindung gefunden wurde

In [7]:
calculate_distance(38.082803, -122.541406, 38.076333, -122.54237)

11732.4

In [5]:
def process_pair(index_pair):
    i, j = index_pair
    if i == j or dis[i, j] > 0:  # Diagonale oder bereits bearbeitet ignorieren
        return
    
    sensor1_id = node[i]
    sensor2_id = node[j]
    try:
        lat1, lng1 = sensor[sensor['station_id'] == sensor1_id][['Lat', 'Lng']].values[0]
        lat2, lng2 = sensor[sensor['station_id'] == sensor2_id][['Lat', 'Lng']].values[0]
    except IndexError:
        print(f"Koordinaten fehlen für Sensoren: {sensor1_id} oder {sensor2_id}, überspringe.")
        return
    
    # Entfernung berechnen
    distance = calculate_distance(lat1, lng1, lat2, lng2)
    if distance is not None:
        dis[i, j] = distance
        dis[j, i] = distance  # Optional, falls Matrix symmetrisch ist
        print(f"Eintrag ({i}, {j}) aktualisiert: {distance:.2f} Meter.")


In [6]:
# Generiere Indexpaare für die Matrix
index_pairs = [
    (i, j)
    for i in range(dis.shape[0])
    for j in range( dis.shape[1])  
]

In [ ]:
max_threads = 16
with ThreadPoolExecutor(max_threads) as executor:
    # Verwende `tqdm` mit as_completed für Fortschritt
    with tqdm(total=len(index_pairs)) as pbar:
        futures = [executor.submit(process_pair, pair) for pair in index_pairs]
        for future in as_completed(futures):
            pbar.update(1)
    

  0%|          | 0/288048784 [00:00<?, ?it/s]

In [29]:
# Funktion zum Abrufen der Distanzmatrix über OSRM Table
def fetch_distance_matrix(locations):
    """
    Holt die Distanzmatrix von OSRM für eine Liste von Koordinaten.
    :param locations: Liste der Koordinaten [(lng, lat), (lng, lat), ...]
    :return: Numpy-Array mit Distanzen
    """
    coords = ";".join([f"{lng},{lat}" for lng, lat in locations])
    url = f"http://192.168.221.197:5000/table/v1/driving/{coords}?annotations=distance"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'distances' in data:
            return np.array(data['distances'])
    return None

In [30]:
output_dir = '/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/node'

In [33]:
# Iteriere über Submatrizen und berechne Distanzen
for split_idx, sub_dis in enumerate(dis_test):
    offset = split_idx * sub_dis.shape[0]
    
    # Extrahiere Sensor-Koordinaten für diese Submatrix
    node_ids = node[offset:offset + sub_dis.shape[0]]
    try:
        locations = sensor[sensor['station_id'].isin(node_ids)][['Lng', 'Lat']].values
    except KeyError:
        print(f"Fehlerhafte Koordinaten für Node IDs in Split {split_idx}")
        continue

    if len(locations) < 2:
        print(f"Nicht genug gültige Koordinaten für Split {split_idx}, überspringe.")
        continue

    # Hole Distanzmatrix
    distance_matrix = fetch_distance_matrix(locations)
    if distance_matrix is not None:
        # Aktualisiere die Submatrix
        np.fill_diagonal(distance_matrix, 0)  # Diagonale auf 0 setzen
        sub_dis[:distance_matrix.shape[0], :distance_matrix.shape[1]] = distance_matrix
    else:
        print(f"Fehler beim Abrufen der Distanzmatrix für Split {split_idx}")

    # Speichere die aktualisierte Teilmatrix
    output_path = os.path.join(output_dir, f"dis_split_{split_idx}.npy")
    np.save(output_path, sub_dis)
    print(f"Teilmatrix {split_idx} gespeichert in {output_path}.")

Teilmatrix 0 gespeichert in /Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/node/dis_split_0.npy.
Teilmatrix 1 gespeichert in /Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/node/dis_split_1.npy.
Teilmatrix 2 gespeichert in /Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/node/dis_split_2.npy.
Teilmatrix 3 gespeichert in /Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/node/dis_split_3.npy.
Teilmatrix 4 gespeichert in /Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/node/dis_split_4.npy.
Teilmatrix 5 gespeichert in /Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/node/dis_split_5.npy.
Teilmatrix 6 gespeichert in /Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/node/dis_split_6.npy.
Teilmatrix 7 gespeichert in /Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/node/dis_split_7.npy.
Teilmatrix 8 ges

In [35]:
output_dir = "/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/node"  # Der Ordner, in dem die Teildistanzmatrizen gespeichert sind
num_splits = 4243  # Anzahl der Splits (sub_dis), hier als Beispiel

# Leere Liste, um alle Teildistanzmatrizen zu speichern
all_dis = []

# Teildistanzmatrizen laden und vertikal zusammenführen
for split_idx in range(num_splits):
    sub_dis_path = os.path.join(output_dir, f"dis_split_{split_idx}.npy")
    
    if os.path.exists(sub_dis_path):
        sub_dis = np.load(sub_dis_path)
        all_dis.append(sub_dis)  # Füge die Teilmatrix der Liste hinzu
        print(f"Teilmatrix {split_idx} geladen.")
    else:
        print(f"Teilmatrix {split_idx} nicht gefunden, überspringe.")

# Vertikal zusammenführen 
full_dis = np.vstack(all_dis)

Teilmatrix 0 geladen.
Teilmatrix 1 geladen.
Teilmatrix 2 geladen.
Teilmatrix 3 geladen.
Teilmatrix 4 geladen.
Teilmatrix 5 geladen.
Teilmatrix 6 geladen.
Teilmatrix 7 geladen.
Teilmatrix 8 geladen.
Teilmatrix 9 geladen.
Teilmatrix 10 geladen.
Teilmatrix 11 geladen.
Teilmatrix 12 geladen.
Teilmatrix 13 geladen.
Teilmatrix 14 geladen.
Teilmatrix 15 geladen.
Teilmatrix 16 geladen.
Teilmatrix 17 geladen.
Teilmatrix 18 geladen.
Teilmatrix 19 geladen.
Teilmatrix 20 geladen.
Teilmatrix 21 geladen.
Teilmatrix 22 geladen.
Teilmatrix 23 geladen.
Teilmatrix 24 geladen.
Teilmatrix 25 geladen.
Teilmatrix 26 geladen.
Teilmatrix 27 geladen.
Teilmatrix 28 geladen.
Teilmatrix 29 geladen.
Teilmatrix 30 geladen.
Teilmatrix 31 geladen.
Teilmatrix 32 geladen.
Teilmatrix 33 geladen.
Teilmatrix 34 geladen.
Teilmatrix 35 geladen.
Teilmatrix 36 geladen.
Teilmatrix 37 geladen.
Teilmatrix 38 geladen.
Teilmatrix 39 geladen.
Teilmatrix 40 geladen.
Teilmatrix 41 geladen.
Teilmatrix 42 geladen.
Teilmatrix 43 geladen

In [36]:
full_dis.shape

(16972, 16972)